In [ ]:
import pickle
import nltk
import random
import spacy
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import os

In [ ]:
!pip install --upgrade pip

# Install spaCy
!pip install -q spacy

# Download and install the English language model
!python -m spacy download en_core_web_sm

# Install VADER
!pip3 install vaderSentiment

# Install WordNet
nltk.download('wordnet')

# Install punkt
nltk.download('punkt')

# Install stopwords
nltk.download('stopwords')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 14.1 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 43.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 3.2 MB/s eta 0:00:00


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [ ]:
with open('knowledge_base.pickle', 'rb') as handle:
  knowledge_base = pickle.load(handle)

for key in knowledge_base.keys():
  print(key, len(knowledge_base[key]))

Stardew Valley 125
ChuckleFish 11
Eric Barone 72
ConcernedApe 8
mobile 8
music 12
multiplayer 15
album 8
android 6
version 10
iOS 11
update 14


In [ ]:
# From some text, extract possible names and randomly choose a name (if multiple names)
def get_name(name_prompt):

  nlp = spacy.load('en_core_web_sm')
  doc = nlp(name_prompt)

  # Initialize an empty list to store potential names
  names = []

  # Extract potential names
  for entity in doc.ents:
    if entity.label_ == "PERSON":
      names.append(entity.text)

  # If no "PERSON" is found then look for patterns in the sentence
  if not names:
    try:
      for token in doc:
        if token.pos_ == "PROPN" and token.dep_ != "compound":
          names.append(entity.text)
    except: # Do nothing
      pass

  if names:
    return random.choice(names)
  else:
    return ""

# Testing
# get_name("Hi there! I love stardew valley")

In [ ]:
# Return the sentiment (positive or negative) of the text

def get_sentiment(text_prompt):

  analyzer = SentimentIntensityAnalyzer()
  vs = analyzer.polarity_scores(text_prompt)
  compound_score = vs['compound']
  return compound_score

# TESTING
#text1 = "Nope"
#print(get_sentiment(text1))

#text2 = "I hate Stardew Valley."
#print(get_sentiment(text2))

#text3 = "Yes. I love stardew valley"
#print(get_sentiment(text3))

#text4 = "Not really."
#print(get_sentiment(text4))

In [ ]:
# MIGHT BE USELESS DELETE LATER

def preprocess_sentence(sentence):

  # Tokenize the raw text and lowercase
  tokenized_words = word_tokenize(sentence.lower())


  # Filter out tokens to tokens that not in the in the stopword list
  stopwords_tokens = [token for token in tokenized_words if token not in stopwords.words('english')]

  # Remove punctuation from the tokens
  punctuation_removed_tokens = [token for token in stopwords_tokens if token not in string.punctuation]

  # Lemmatize the tokens
  lemmatizer = WordNetLemmatizer()
  lemmatized_tokens = [lemmatizer.lemmatize(token) for token in punctuation_removed_tokens]

  return lemmatized_tokens

# Testing
tokens = preprocess_sentence("Is he lonely?")
#print(tokens)
#print(sentences)

In [ ]:
def calculate_similarity(sentence, word_phrase):
  # Create corpus
  corpus = [sentence, word_phrase]

  # Create vectorizer
  tfidf_vec = TfidfVectorizer()

  # Transform the corpus
  tfidf_matrix = tfidf_vec.fit_transform(corpus)

  # Calculate cosine similarity between the sentence and word vectors
  similarity = cosine_similarity(tfidf_matrix[0], tfidf_matrix[1])

  return similarity[0][0]

# Example usage
sentence1 = "I want to know more about the music of stardew valley."
word1 = "ChuckleFish"
similarity1 = calculate_similarity(sentence1, word1)
#print("Similarity:", similarity1)

sentence2 = "I want to know more about the music of stardew valley."
word2 = "Music"
similarity2 = calculate_similarity(sentence2, word2)
#print("Similarity:", similarity2)

In [ ]:
# All the keys(terms) of our knowledge base
terms_list = list(knowledge_base.keys())

# Try to determine the best term from the prompt
def get_term_from_prompt(prompt, terms_list):
  # Naive and easy way to calculate max (but this isn't an algorithms class so we good)
  similarity = 0
  similar_term = ""

  for term in terms_list:
    current_sim = calculate_similarity(prompt, term)
    if current_sim > similarity:
      similarity = current_sim
      similar_term = term

  return similar_term

# Testing
best_term = get_term_from_prompt("", terms_list)
#print(best_term)

best_term = get_term_from_prompt("Is Stardew Valley open-ended?", terms_list)
#print(best_term)

In [ ]:
# Try to determine the best sentence from a prompt
def get_sentence_from_prompt(prompt, term):
  # Naive and easy way to calculate max (but this isn't an algorithms class so we good)
  similarity = 0
  similar_sentence = ""

  try:
    sentence_list = knowledge_base[term]
    for sentence in sentence_list:
      current_sim = calculate_similarity(prompt, sentence)
      if current_sim > similarity:
        similarity = current_sim
        similar_sentence = sentence

    return similar_sentence

  except:
    return ""
# Testing
best_sentence = get_sentence_from_prompt("Is Eric Barone's alone?", "Eric Barone")
#print(best_sentence)

In [ ]:
def process_name_to_filename(name):
  filename = name.replace(" ", "_")
  #filename += ".txt"
  return filename

In [ ]:
def get_like_or_dislikes_nouns(text):
  nlp = spacy.load('en_core_web_sm')

  doc = nlp(text)

    # Initialize an empty list to store potential names
  nouns = []

  # Extract potential names
  for token in doc:
    #print(token.text, token.pos_, token.dep_)
    if token.pos_ == 'NOUN' and (token.dep_ == 'ROOT' or token.dep_ ==  'attr' or token.dep_ == 'dobj' or token.dep_ == 'nmod' or token.dep_ == 'conj'):
      nouns.append(token.text)
      #print(token.text, token.pos_, token.dep_)

  return nouns

# Testing

#print(get_like_or_dislikes_nouns("I like the music"))
#print(get_like_or_dislikes_nouns("I hate the music"))
#print(get_like_or_dislikes_nouns("I like the music, gameplay and the art"))
#print(get_like_or_dislikes_nouns("I like the music"))

In [ ]:
def get_like_or_dislikes_adj(text):
  nlp = spacy.load('en_core_web_sm')

  doc = nlp(text)

    # Initialize an empty list to store potential names
  jj = []

  # Extract potential names
  for token in doc:
    #print(token.text, token.pos_, token.dep_)
    if token.pos_ == 'ADJ' and (token.dep_ == 'acomp' or token.dep_ == 'conj' ):
      jj.append(token.text)
      #print(token.text, token.pos_, token.dep_)

  return jj

# Testing

#print(get_like_or_dislikes_adj("It's slow and boring."))

In [ ]:
import string

yes_synonyms = ['positive', 'yes', 'yeah', 'sure', 'certainly', 'indeed', 'absolutely', 'yup', 'ok', 'yep', 'yah', 'yea', 'of course', 'all right']
no_synonyms = ['negative', 'nay', 'nope', 'nah', 'not at all', 'never', 'no', 'not really']

def get_yes_or_no(text):
  # Lower case the text
  text = text.lower()

  # Remove punctuation
  text = ''.join(char for char in text if char not in string.punctuation)

  # Check if any entire phrase is in yes_synonyms or no_synonyms
  for phrase in yes_synonyms:
    if phrase in text:
      return 1

  for phrase in no_synonyms:
    if phrase in text:
      return -1

  # Neutral
  return 0

# TESTING
text = "not really"
response = get_yes_or_no(text)
#print(response)


In [ ]:
console_list = ['iOS', 'Android', 'Windows', 'macOS', 'Linux', 'PlayStation 4',
               'Xbox One', 'Nintendo Switch', "Playstation Vita", "PC"]

# Returns a list of consoles from a sentence
def get_consoles(sentence, console_list = ['iOS', 'android', 'Windows', 'macOS', 'Linux', 'PlayStation 4',
               'Xbox One', 'Nintendo Switch', "Playstation Vita", "PC"]):

# Check if the sentence contains any console names
  found_consoles = [console for console in console_list if console.lower() in sentence.lower()]

  return found_consoles

# Testing
#sentence = "I love playing games on my PlayStation 4 and Nintendo Switch."
#get_consoles(sentence, console_list)

In [ ]:
def get_random_fact(term):
  corpus = knowledge_base[term]
  random_sentence = random.choice(corpus)
  return random_sentence

In [ ]:
## START OF CHATBOT RULES/LOGIC

# Hard coded some greetings
greetings = [
  "Hello",
  "Greetings",
  "Pleased to meet you",
  "Hi",
  "Hey there"
]

# Hard coded some farewells
farewells = [
  "Goodbye",
  "See you later",
  "Farewells",
  "Nice talking to you"
]

#Chatbot name
chatbot_name = "Haley-Abigail"
chatbot_onscreen = chatbot_name + ": "

#Initial username (will be changed later)
user_name = "You"
user_name_onscreen = "\n" + user_name + ": "

like_set = set()
dislike_set = set()

user_model = {}

# Choose a random string from a list of strings
def choose_random(list_string):
  return random.choice(list_string)


initial_prompt = "I'm %s, a fan-bot of Stardew Valley. I love to talk about Stardew Valley! What is your name? %s" % (chatbot_name, user_name_onscreen)

user_response = input(chatbot_onscreen + choose_random(greetings) + "! " + initial_prompt)
user_name = get_name(user_response)

# Case where a name is not given/found
while not user_name or (user_name == chatbot_name):
  user_response = input(chatbot_onscreen + "Sorry about that, I couldn't get your name. Can you repeat your name please? %s" % user_name_onscreen)
  user_name = get_name(user_response)

# Update the username on screen
user_name_onscreen = "\n" + user_name + ": "

# Store name in the user_model
user_model["name"] = user_name

# Initialize the pickle name
user_picklename = process_name_to_filename(user_name) + ".pickle"
user_pickle_folder = 'user_models/'
user_pickle_path = user_pickle_folder + user_picklename
# Check if the folder exists
if not os.path.exists(user_pickle_folder):
  # If it doesn't exist, create it
  os.makedirs(user_pickle_folder)

# Case if the chatbot has talked this person before
if os.path.exists(user_pickle_path):
  with open(user_pickle_path, 'rb') as handle:
    stored_user_model = pickle.load(handle)
    # print(stored_user_model)
  # Get the name
  if "name" in stored_user_model:
    print(chatbot_onscreen + "Welcome back %s!" %(stored_user_model["name"]))

  # Gets if the user likes/dislikes Stardew
  if "likes_Stardew" in stored_user_model:
    # User likes stardew
    if stored_user_model["likes_Stardew"]:
      print(chatbot_onscreen + "From our last conversation, I remembered that you liked Stardew Valley.")

    # User dislikes stardew
    if not stored_user_model["likes_Stardew"]:
      print(chatbot_onscreen + "From our last conversation, I remembered that you disliked Stardew Valley.")
      response = input(chatbot_onscreen + "Do you still dislike Stardew Valley %s? %s" % (user_name, user_name_onscreen))
      sentiment = get_sentiment(response)

      while not sentiment:
        # Get yes or no
        if sentiment == 0:
          sentiment = get_yes_or_no(response)

        if sentiment:
          break

        print(chatbot_onscreen + "Sorry about that, I couldn't get your response.")
        response = input(chatbot_onscreen + "Do you still dislike Stardew Valley %s? %s" % (user_name, user_name_onscreen))
        sentiment = get_sentiment(response)

      # No they like Stardew now
      if sentiment < 0:
        stored_user_model["likes_Stardew"] = True

  if "likes" in stored_user_model:
    likes = stored_user_model["likes"]
    if len(likes) != 0:
      joined_likes = ' and '.join(likes)
      print(chatbot_onscreen + "When we talked last, I remembered that you liked the %s about Stardew Valley." %(joined_likes))

    response = input(chatbot_onscreen + "Are there any additional likes about Stardew Valley since our last conversation %s? %s" % (user_name, user_name_onscreen))
    sentiment = get_sentiment(response)

    while not sentiment:
      # Get yes or no
      if sentiment == 0:
        sentiment = get_yes_or_no(response)

      if sentiment:
        break

      print(chatbot_onscreen + "Sorry about that, I couldn't get your response.")
      response = input(chatbot_onscreen + "Are there any additional likes about Stardew Valley %s? %s" % (user_name, user_name_onscreen))
      sentiment = get_sentiment(response)

    if sentiment > 0:
      current_likes = get_like_or_dislikes_nouns(response)

      adj_flag = False
      if not current_likes:
        current_likes = get_like_or_dislikes_adj(response)
        if current_likes:
          adj_flag = True

      # Could not process the prompt
      while not current_likes:
        response = input(chatbot_onscreen + "Sorry about that, I couldn't get your likes. Can you repeat your likes please? %s" % user_name_onscreen)
        current_likes = get_like_or_dislikes_nouns(response)
        if not current_likes:
          current_likes = get_like_or_dislikes_adj(response)
          if current_likes:
            adj_flag = True

      likes.update(current_likes)
      likes_string = " and ".join(current_likes)

      stored_user_model["likes"] = likes
      # Relay back to User
      if adj_flag:
        print(chatbot_onscreen + "You additionally like Stardew Valley because its %s." % (likes_string))
      else:
        print(chatbot_onscreen + "You additionally like %s about Stardew Valley." % (likes_string))

  if "dislikes" in stored_user_model:
    dislikes = stored_user_model["dislikes"]
    if len(dislikes) != 0:
      joined_dislikes = ' and '.join(dislikes)
      print(chatbot_onscreen + "I also remembered that you disliked the %s about Stardew Valley." %(joined_dislikes))

    response = input(chatbot_onscreen + "Are there any additional dislikes about Stardew Valley since our last conversation %s? %s" % (user_name, user_name_onscreen))
    sentiment = get_sentiment(response)
    if sentiment == 0:
      sentiment = -1 * get_yes_or_no(response)

    while not sentiment:
      # Get yes or no
      if sentiment == 0:
        sentiment = -1 * get_yes_or_no(response)

      if sentiment:
        break

        print(chatbot_onscreen + "Sorry about that, I couldn't get your response.")
        response = input(chatbot_onscreen + "Are there any additional dislikes about Stardew Valley %s? %s" % (user_name, user_name_onscreen))
        sentiment = get_sentiment(response)

    if sentiment < 0:
      current_dislikes = get_like_or_dislikes_nouns(response)

      adj_flag_dislike = False
      if not current_dislikes:
        current_dislikes = get_like_or_dislikes_adj(response)
        if current_dislikes:
          adj_flag_dislike = True

      # Could not process the prompt
      while not current_dislikes:
        response = input(chatbot_onscreen + "Sorry about that, I couldn't get your dislikes. Can you repeat your dislikes please? %s" % user_name_onscreen)
        current_dislikes = get_like_or_dislikes_nouns(response)
        if not current_dislikes:
          current_dislikes = get_like_or_dislikes_adj(response)
          if current_dislikes:
            adj_flag_dislike = True

      dislikes.update(current_dislikes)
      dislikes_string = " and ".join(current_dislikes)

      stored_user_model["dislikes"] = dislikes
      # Relay back to User
      if adj_flag_dislike:
        print(chatbot_onscreen + "You additionally dislike Stardew Valley because its %s." % (dislikes_string))
      else:
        print(chatbot_onscreen + "You additionally dislike %s about Stardew Valley." % (dislikes_string))


  # This entire block asks if the user has played Stardew since the last time the chatbot talked
  if "played_Stardew" in stored_user_model:
    if stored_user_model["played_Stardew"]:
      try:
        platforms_played = stored_user_model["platforms_played"]
        platforms_played_string = " or ".join(platforms_played)
        question = "Have you played Stardew Valley on %s since the last time we talked %s? %s" % (platforms_played_string, user_name, user_name_onscreen)

      except:
        question = "Have you played Stardew Valley since the last time we talked %s? %s" % (user_name, user_name_onscreen)


    if not stored_user_model["played_Stardew"]:
      try:
        platforms_owned = stored_user_model["platforms_owned"]
        platforms_owned_string = " or ".join(platforms_owned)
        question = "Have you played Stardew Valley on %s since the last time we talked %s? %s" % (platforms_played_string, user_name, user_name_onscreen)
      except:
        question = "Have you played Stardew Valley since the last time we talked %s? %s" % (user_name, user_name_onscreen)

    response = input(chatbot_onscreen + question)
    sentiment = get_sentiment(response)

    while not sentiment:
      if sentiment == 0:
        sentiment = get_yes_or_no(response)

      if sentiment:
        break

      print(chatbot_onscreen + "Sorry about that, I couldn't get your response.")
      response = input(chatbot_onscreen + question)
      sentiment = get_sentiment(response)

    # Update that the user has played Stardew
    if sentiment > 0:
      stored_user_model["played_Stardew"] = True



  with open(user_pickle_path, 'wb') as handle:
    pickle.dump(stored_user_model, handle)
# ----------------------------------
# GET LIKES OR DISLIKES
# ----------------------------------
else:
  response = input(chatbot_onscreen + choose_random(greetings) + " %s. Do you like Stardew Valley? %s" % (user_name, user_name_onscreen))
  sentiment = get_sentiment(response)

  while not sentiment:
    # Get yes or no
    if sentiment == 0:
      sentiment = get_yes_or_no(response)

    if sentiment:
      break

    print(chatbot_onscreen + "Sorry about that, I couldn't get your response.")
    response = input(chatbot_onscreen + choose_random(greetings) + " %s. Do you like Stardew Valley? %s" % (user_name, user_name_onscreen))
    sentiment = get_sentiment(response)

  # Case where the response is positive
  if sentiment > 0:
    # Store fact that the user like Stardew
    user_model["likes_Stardew"] = True

    # Ask the likes
    print(chatbot_onscreen + "I love Stardew Valley as well!")
    response = input(chatbot_onscreen + "What do you like about Stardew Valley?%s" % (user_name_onscreen))

    sentiment2 = get_sentiment(response)

    if sentiment2 > 1:
      # Get the likes of the user
      current_likes = get_like_or_dislikes_nouns(response)

      adj_flag = False
      if not current_likes:
        current_likes = get_like_or_dislikes_adj(response)
        if current_likes:
          adj_flag = True

      # Could not process the prompt
      while not current_likes:
        response = input(chatbot_onscreen + "Sorry about that, I couldn't get your likes. Can you repeat your likes please? %s" % user_name_onscreen)
        current_likes = get_like_or_dislikes_nouns(response)
        if not current_likes:
          current_likes = get_like_or_dislikes_adj(response)
          if current_likes:
            adj_flag = True

      like_set.update(current_likes)
      likes_string = " and ".join(current_likes)

      # Relay back to User
      if adj_flag:
        print(chatbot_onscreen + "You like because its %s as well? Me too! Honestly, I love everything about Stardew Valley." % (likes_string))
      else:
        print(chatbot_onscreen + "You like its %s as well? Me too! Honestly, I love everything about Stardew Valley." % (likes_string))




  # Case where the response is negative
  elif sentiment < 0:

    # Store fact that the user dislikes Stardew
    user_model["likes_Stardew"] = False


    response = input(chatbot_onscreen + "Really? That's too bad. Why don't you like Stardew Valley? %s" % (user_name_onscreen))
    sentiment2 = get_sentiment(response)

    if sentiment2 < 1:
    # Get the dislikes of the user
      current_dislikes = get_like_or_dislikes_nouns(response)

      # Get adjectives if nouns not found
      adj_flag = False
      if not current_dislikes:
        current_dislikes = get_like_or_dislikes_adj(response)
        if current_dislikes:
          adj_flag = True

      # Could not process the prompt
      while not current_dislikes:
        response = input(chatbot_onscreen + "Sorry about that, I couldn't get your dislikes. Can you repeat your dislikes please? %s" % user_name_onscreen)
        current_dislikes = get_like_or_dislikes_nouns(response)
        if not current_dislikes:
          current_dislikes = get_like_or_dislikes_adj(response)
          if current_dislikes:
            adj_flag = True


      # Update dislike set
      dislike_set.update(current_dislikes)
      dislikes_string = " and ".join(current_dislikes)

      # Relay back to User
      if adj_flag:
        print(chatbot_onscreen + "I see... You dislike because its %s." % (dislikes_string))
      else:
        print(chatbot_onscreen + "I see... You dislike its %s." % (dislikes_string))

  # ----------------------------------------------------
  # ASK ABOUT IF USER HAS KNOWS ConcernedApe
  # ----------------------------------------------------
  response = input(chatbot_onscreen + "Do you know who ConcernedApe, otherwise known as Eric Barone, is? %s" % (user_name_onscreen))
  sentiment = get_yes_or_no(response)

  while not sentiment:
    # Couldn't process prompt
    print(chatbot_onscreen + "Sorry about that, I couldn't get your response.")
    response = input(chatbot_onscreen + "Do you know who ConcernedApe, otherwise known as Eric Barone, is? %s" % (user_name_onscreen))
    sentiment = get_yes_or_no(response)

  # Yes case
  if sentiment > 0:
    print(chatbot_onscreen + "Yea! He's the sole creator of Stardew Valley! Here's a random fact about him: ")
    print(chatbot_onscreen + get_random_fact("Eric Barone"))

  # No case
  if sentiment < 0:
    print(chatbot_onscreen + "He's the sole creator of Stardew Valley!")
    response = input(chatbot_onscreen + "What would you like to know about Eric Barone. %s" % (user_name_onscreen))
    fact = get_sentence_from_prompt(user_response, "Eric Barone")
    if not fact:
      fact = get_sentence_from_prompt(user_response, "ConcernedApe")
    if not fact:
      fact = get_random_fact("Eric Barone")
    print(chatbot_onscreen + fact)

  # ----------------------------------
  # ASK ABOUT ChuckleFish
  # ----------------------------------
  response = input(chatbot_onscreen + "Do you know about ChuckleFish? %s" % (user_name_onscreen))
  sentiment = get_yes_or_no(response)

  while not sentiment:
    # Couldn't process prompt
    print(chatbot_onscreen + "Sorry about that, I couldn't get your response.")
    response = input(chatbot_onscreen + "Do you know about ChuckleFish? %s" % (user_name_onscreen))
    sentiment = get_sentiment(response)

  # Yes case
  if sentiment > 0:
    print(chatbot_onscreen + "Here's a random fact about ChuckleFish: ")
    print(chatbot_onscreen + get_random_fact("ChuckleFish"))

  # No case
  if sentiment < 0:
    print(chatbot_onscreen + "It was the original publisher of Stardew Valley.")
    response = input(chatbot_onscreen + "What would you like to know about ChuckleFish. %s" % (user_name_onscreen))
    fact = get_sentence_from_prompt(response, "ChuckleFish")
    if not fact:
      fact = get_random_fact("ChuckleFish")
    print(chatbot_onscreen + fact)


  # ----------------------------------------------------
  # ASK ABOUT IF USER HAS EVER PLAYED STARDEW VALLEY
  # ----------------------------------------------------
  response = input(chatbot_onscreen + "Have you ever played Stardew Valley? %s" % (user_name_onscreen))
  sentiment = get_yes_or_no(response)

  while not sentiment:
    print(chatbot_onscreen + "Sorry about that, I couldn't get your response.")
    response = input(chatbot_onscreen + "Have you ever played Stardew Valley? %s" % (user_name_onscreen))
    sentiment = get_sentiment(response)

  # Yes case for Played Stardew in the past
  if sentiment > 0:
    # Store the fact that the user played Stardew
    user_model["played_Stardew"] = True
    response = input(chatbot_onscreen + "Which console did you play it on?%s" % (user_name_onscreen))
    # print(chatbot_onscreen + get_sentence_from_prompt(user_response, "version"))
    platforms = get_consoles(response)

    # Case when platforms is not processed
    while not platforms:
      print(chatbot_onscreen + "Sorry about that, I couldn't get your response.")
      response = input(chatbot_onscreen + "Which console did you play it on?%s" % (user_name_onscreen))
      platforms = get_consoles(response)

    # Store the platforms played
    user_model["platforms_played"] = platforms

    platforms_string = " and ".join(platforms)
    print(chatbot_onscreen + "You played on the %s platform(s)." % platforms_string)

    mobile_displayed_flag = False
    for platform in platforms:
      if platform.lower() in ("ios", "android"):
        # Makes sure that this is only processed once per for loop
        if not mobile_displayed_flag:
          print(chatbot_onscreen + "You played on Stardew Valley on mobile devices.")
          print(chatbot_onscreen + "Here's a random fact about Stardew Valley on mobile devices: ")
          print(chatbot_onscreen + get_random_fact("mobile"))
          mobile_displayed_flag = True

        print(chatbot_onscreen + "Here's fun fact about Stardew Valley on %s: " %(platform))
        print(chatbot_onscreen + get_random_fact(platform))


    # Ask if the user has played Multiplayer
    # Get yes or no
    response = input(chatbot_onscreen + "Have you ever played Stardew Valley on multiplayer? %s" % (user_name_onscreen))
    sentiment2 = get_yes_or_no(response)

    while not sentiment2:
      print(chatbot_onscreen + "Sorry about that, I couldn't get your response.")
      response2 = input(chatbot_onscreen + "Have you ever played Stardew Valley on multiplayer? %s" % (user_name_onscreen))
      sentiment2 = get_sentiment(response)

    # Yes case for played multiplayer
    if sentiment2 > 0:
      user_model["played_multiplayer"] = True

    # No case for played multiplayer
    if sentiment2 < 0:
      user_model["played_multiplayer"] = False
      print(chatbot_onscreen + "It's very fun with a couple of friends!")

    print(chatbot_onscreen + "Here's a random fact about the multiplayer: ")
    print(chatbot_onscreen + get_random_fact("multiplayer"))

  # No case
  if sentiment < 0:
    user_model["played_Stardew"] = False
    print(chatbot_onscreen + "You should play it!")
    response = input(chatbot_onscreen + "Stardew Valley is on many platforms. Do you want to know which ones? %s" % (user_name_onscreen))

    sentiment2 = get_yes_or_no(response)
    while not sentiment2:
      print(chatbot_onscreen + "Sorry about that, I couldn't get your response.")
      response = input(chatbot_onscreen + "Stardew Valley is on many platforms. Do you want to know which ones?  %s" % (user_name_onscreen))
      sentiment2 = get_sentiment(response)

    if sentiment2 > 0:
      console_string_to_print = ', '.join(console_list[:-1]) + ', and ' + console_list[-1]
      print(chatbot_onscreen + "Stardew Valley is available on %s." %(console_string_to_print))

    response = input(chatbot_onscreen + "Do you own one of these platforms?%s" % (user_name_onscreen))
    sentiment3 = get_yes_or_no(response)

    while not sentiment3:
      print(chatbot_onscreen + "Sorry about that, I couldn't get your response.")
      response = input(chatbot_onscreen + "Do you own one of these platforms? %s" % (user_name_onscreen))
      sentiment3 = get_sentiment(response)


    if sentiment3 > 0:
      response = input(chatbot_onscreen + "Which platforms do you own?%s" % (user_name_onscreen))
      platforms = get_consoles(response)
      while not platforms:
        print(chatbot_onscreen + "Sorry about that, I couldn't get your response.")
        response = input(chatbot_onscreen + "Which platforms do you own?%s" % (user_name_onscreen))
        platforms = get_consoles(response)

      # Store the platforms played
      user_model["platforms_owned"] = platforms
      user_platforms_owned = ' or '.join(platforms)
      print(chatbot_onscreen + "You should play Stardew Valley on %s." %(user_platforms_owned))


  # ----------------------------------------------------
  # ASK ABOUT IF USER HAS LIKES THE MUSIC
  # ----------------------------------------------------
  response = input(chatbot_onscreen + "Do you like the music of Stardew Valley? %s" % (user_name_onscreen))
  sentiment = get_sentiment(response)

  while not sentiment:
    # Get yes or no
    if sentiment == 0:
      sentiment = get_yes_or_no(response)

    if sentiment:
      break

    print(chatbot_onscreen + "Sorry about that, I couldn't get your response.")
    response = input(chatbot_onscreen + "Do you like the music of Stardew Valley? %s" % (user_name_onscreen))
    sentiment = get_sentiment(response)

  # Yes case
  if sentiment > 0:
    # Store the fact the user likes the music
    like_set.add("music")


    print(chatbot_onscreen + "Here's a random fact about its music: ")
    print(chatbot_onscreen + get_random_fact("music"))

    response = input(chatbot_onscreen + "Did you know that Stardew Valley also has an album?%s" % (user_name_onscreen))

    #response doesn't matter here (its fake)
    print(chatbot_onscreen + get_random_fact("album"))



  # No case
  if sentiment < 0:
    # Store the fact the user dislikes the music
    dislike_set.add("music")
    print(chatbot_onscreen + "You should give it another chance!")
    print(chatbot_onscreen + "They even have a physical vinyl album!")

  # Adding likes/dislikes to the user model
  user_model["likes"] = like_set
  user_model["dislikes"] = dislike_set

  with open(user_pickle_path, 'wb') as handle:
    pickle.dump(user_model, handle)

Haley-Abigail: Greetings! I'm Haley-Abigail, a fan-bot of Stardew Valley. I love to talk about Stardew Valley! What is your name? 
You: Hi there Haley-Abigail
Haley-Abigail: Sorry about that, I couldn't get your name. Can you repeat your name please? 
You: My name is John
Haley-Abigail: Pleased to meet you John. Do you like Stardew Valley? 
John: Yes
Haley-Abigail: I love Stardew Valley as well!
Haley-Abigail: What do you like about Stardew Valley?
John: I don't like the music


KeyboardInterrupt: Interrupted by user

In [ ]:
# Ran out of conversation topics, will repeat this until the end.

print(chatbot_onscreen + "I have ran out of questions for you." )
print(chatbot_onscreen + "Now it's your turn to ask!")
print(chatbot_onscreen + "These are the following terms I know alot about: " )
print(chatbot_onscreen + "Type in 'bye' to end this conversation" )

# Printing the terms
for term in terms_list:
  print("  * " + term)

while(True):
  user_response = input(chatbot_onscreen + "What would you like to know about one of the terms?%s" % (user_name_onscreen))
  if user_response.lower() == ('bye' or 'goodbye'):
    break
  term_from_user = get_term_from_prompt(user_response, terms_list)
  if not term_from_user:
    term_from_user = "Stardew Valley"

  answer = get_sentence_from_prompt(user_response, term_from_user)
  if not answer:
    print(chatbot_onscreen + "Sorry. I couldn't understand your question")
  else:
    print(chatbot_onscreen + answer)

print(chatbot_onscreen + choose_random(farewells) + ".")